In [3]:
import pandas as pd
import mysql.connector
from datetime import datetime



# List of CSV file paths["bus_det_final_csvfile_2_kaac.csv"]
csv_files = ["ahemedabad.csv","Andhra_bus_details.csv","bangalore_bus_details.csv","chennai_bus_details.csv","Himachal_bus_details.csv","Kerala_bus_details.csv","pune_bus_routes.csv","rajasthan_bus_details.csv","telungana_bus_details.csv","uttarpradesh_bus_details.csv"]

# Read each CSV file into a DataFrame and concatenate them
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].fillna(0).astype(int)
combined_df = combined_df.dropna(subset=['Bus_Name'])
combined_df = combined_df.dropna(subset=['Bus_Type'])



# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="RED_BUS"
)

mycursor = mydb.cursor()






# Create table query
create_table_query = """CREATE TABLE RED_BUS_ROUTES(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time TIME NOT NULL,
    duration TEXT NOT NULL,
    reaching_time TIME NOT NULL,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
    """

mycursor.execute(create_table_query)

# Insert query (ensure that the column names match exactly with the DataFrame)
insert_query = """
INSERT INTO RED_BUS_ROUTES (id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame
for i, row in combined_df.iterrows():
    data = (
        row['id'],
        row['Route_Name'],
        row['Route_Link'],
        row['Bus_Name'],
        row['Bus_Type'],
        row['Departing_Time'],
        row['Duration'],
        row['Reaching_Time'],
        row['Star_Rating'],
        row['Price'],
        row['Seat_Availability']
    )
    mycursor.execute(insert_query, data)

# Commit the changes
mydb.commit()

print("Data inserted successfully.")


ProgrammingError: 1050 (42S01): Table 'red_bus_routes' already exists

In [13]:
print(combined_df[['Departing_Time', 'Reaching_Time']].head(10))


  Departing_Time Reaching_Time
0          11:30         16:00
1          12:00         16:45
2          12:30         17:00
3          12:30         17:45
4          13:00         18:00
5          13:15         17:45
6          14:00         19:00
7          14:45         19:45
8          15:00         19:15
9          15:15         20:15


In [5]:
import pandas as pd
import mysql.connector
from datetime import datetime

# List of CSV file paths
csv_files = ["ahemedabad.csv", "Andhra_bus_details.csv", "bangalore_bus_details.csv", "chennai_bus_details.csv",
             "Himachal_bus_details.csv", "Kerala_bus_details.csv", "pune_bus_routes.csv", "rajasthan_bus_details.csv",
             "telungana_bus_details.csv", "uttarpradesh_bus_details.csv"]

# Read each CSV file into a DataFrame and concatenate them
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].fillna(0).astype(int)
combined_df = combined_df.dropna(subset=['Bus_Name'])
combined_df = combined_df.dropna(subset=['Bus_Type'])

# Define function to convert 24-hour time to 12-hour format with AM/PM
def convert_railway_time(railway_times):
    standard_times = []
    for time in railway_times:
        try:
            # Convert railway time to datetime object
            dt = datetime.strptime(time, "%H:%M")
            # Format it to 12-hour clock with AM/PM
            standard_time = dt.strftime("%I:%M %p")
        except ValueError:
            # If there's an error (invalid format), append None
            standard_time = None
        standard_times.append(standard_time)
    return standard_times

# Apply the conversion function to Departing_Time and Reaching_Time columns
combined_df['Departing_Time'] = convert_railway_time(combined_df['Departing_Time'])
combined_df['Reaching_Time'] = convert_railway_time(combined_df['Reaching_Time'])

# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="RED_BUS"
)

mycursor = mydb.cursor()

# Create table query
create_table_query = """CREATE TABLE RED_BUS_ROUTES(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time TIME NOT NULL,
    duration TEXT NOT NULL,
    reaching_time TIME NOT NULL,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
    """
mycursor.execute(create_table_query)

# Insert query
insert_query = """
INSERT INTO RED_BUS_ROUTES (id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


# Commit the changes
mydb.commit()

print("Data inserted successfully.")


Data inserted successfully.


In [3]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Function to convert 24-hour time format to 12-hour format with AM/PM
def convert_to_12_hour(time_str):
    if pd.isna(time_str):
        return None
    try:
        return datetime.strptime(time_str, "%H:%M").strftime("%I:%M %p")
    except ValueError:
        return None

# List of CSV file paths
csv_files = ["ahemedabad.csv", "Andhra_bus_details.csv", "bangalore_bus_details.csv",
             "chennai_bus_details.csv", "Himachal_bus_details.csv", "Kerala_bus_details.csv",
             "pune_bus_routes.csv", "rajasthan_bus_details.csv", "telungana_bus_details.csv", 
             "uttarpradesh_bus_details.csv"]

# Read each CSV file into a DataFrame and concatenate them
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].fillna(0).astype(int)

# Drop rows with missing Bus_Name or Bus_Type
combined_df = combined_df.dropna(subset=['Bus_Name', 'Bus_Type'])

# Drop rows where Departing_Time or Reaching_Time are missing
combined_df = combined_df.dropna(subset=['Departing_Time', 'Reaching_Time'])

# Convert Departing_Time and Reaching_Time from 24-hour to 12-hour format
combined_df['Departing_Time'] = combined_df['Departing_Time'].apply(convert_to_12_hour)
combined_df['Reaching_Time'] = combined_df['Reaching_Time'].apply(convert_to_12_hour)

# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="RED_BUS"
)

mycursor = mydb.cursor()

# Create table query
create_table_query = """CREATE TABLE IF NOT EXISTS RED_BUS_ROUTES8(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time VARCHAR(8) NOT NULL,
    duration TEXT NOT NULL,
    reaching_time VARCHAR(8) NOT NULL,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
    """

mycursor.execute(create_table_query)

# Insert query (ensure that the column names match exactly with the DataFrame)
insert_query = """
INSERT INTO RED_BUS_ROUTES8 (id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame
for i, row in combined_df.iterrows():
    data = (
        row['id'],
        row['Route_Name'],
        row['Route_Link'],
        row['Bus_Name'],
        row['Bus_Type'],
        row['Departing_Time'],
        row['Duration'],
        row['Reaching_Time'],
        row['Star_Rating'],
        row['Price'],
        row['Seat_Availability']
    )
    mycursor.execute(insert_query, data)

# Commit the changes
mydb.commit()

print("Data inserted successfully.")


EmptyDataError: No columns to parse from file

In [5]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Function to convert 24-hour time format to 12-hour format with AM/PM
def convert_to_12_hour(time_str):
    if pd.isna(time_str):
        return None
    try:
        return datetime.strptime(time_str, "%H:%M").strftime("%I:%M %p")
    except ValueError:
        return None

# List of CSV file paths
csv_files = ["ahemedabad.csv", "Andhra_bus_details.csv", "bangalore_bus_details.csv",
             "chennai_bus_details.csv", "Himachal_bus_details.csv", "Kerala_bus_details.csv",
             "pune_bus_routes.csv", "rajasthan_bus_details.csv", "telungana_bus_details.csv", 
             "uttarpradesh_bus_details.csv"]

# Read each CSV file into a DataFrame and concatenate them
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].fillna(0).astype(int)

# Drop rows with missing Bus_Name or Bus_Type
combined_df = combined_df.dropna(subset=['Bus_Name', 'Bus_Type'])

# Drop rows where Departing_Time or Reaching_Time are missing
combined_df = combined_df.dropna(subset=['Departing_Time', 'Reaching_Time'])

# Convert Departing_Time and Reaching_Time from 24-hour to 12-hour format
combined_df['Departing_Time'] = combined_df['Departing_Time'].apply(convert_to_12_hour)
combined_df['Reaching_Time'] = combined_df['Reaching_Time'].apply(convert_to_12_hour)

# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="RED_BUS"
)

mycursor = mydb.cursor()

# Create table query
create_table_query = """CREATE TABLE IF NOT EXISTS RED_BUS_ROUTES8_new(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time VARCHAR(8) NOT NULL,
    duration TEXT NOT NULL,
    reaching_time VARCHAR(8) NOT NULL,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
    """

mycursor.execute(create_table_query)

# Insert query (ensure that the column names match exactly with the DataFrame)
insert_query = """
INSERT INTO RED_BUS_ROUTES_new (id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame
for i, row in combined_df.iterrows():
    data = (
        row['id'],
        row['Route_Name'],
        row['Route_Link'],
        row['Bus_Name'],
        row['Bus_Type'],
        row['Departing_Time'],
        row['Duration'],
        row['Reaching_Time'],
        row['Star_Rating'],
        row['Price'],
        row['Seat_Availability']
    )
    mycursor.execute(insert_query, data)

# Commit the changes
mydb.commit()

print("Data inserted successfully.")


EmptyDataError: No columns to parse from file